# Assignment 3

A sequence of `n` operations is performed on a data structure. The ith operation costs `i` if `i` is an exact power of `2`, and `1` otherwise. That is, operation `i` costs `f(i)`, where `f(i) = i`, if `i = 2^(k)` for some `k ≥ 0`, and `f(i) = 1` otherwise. Determine the amortized cost per operation using the following methods of analysis:

(a) Aggregate method

(b) Accounting method

(c) Potential method

In [53]:
def is_power_of_two(n):
    if n == 0:
        return False
    while n != 1:
        if n % 2 != 0:
            return False
        n = n // 2
    return True

count = []
sum_ones = 0
sum_non_ones = 0
for i in range(1,33):
    if is_power_of_two(i):
        count.append(i)
        sum_non_ones += i
    else:
        count.append(1)
        sum_ones += 1

fh = count[:len(count)//2 ]
sh = count[len(count)//2:]
print(" | ".join(map(str, fh)))
print(" | ".join(map(str, sh)))
print(sum_ones, sum_non_ones)

1 | 2 | 1 | 4 | 1 | 1 | 1 | 8 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 16
1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 32
26 63


## Aggregate 

For an `n` of 32, we see that we have a cost of `26` of the inexpensive `O(1)` operations, and `63` of expensive operations. We can observe that expensive operations cost approximately `2n`, and that non expensive operations cost `n - e_o` where `e_o` is the expensive operations. Reducing those approximates we see that the total cost would be `2n + n` for any `n` operations. The average cost of `3n` over `n` elements would be `3`, which means the aggregate result for this data structure is `O(3)` which is `O(1)`.

## Accounting

If we divide the total cost of an `n` of `32` we will observe an `89`, if we divide that by the number of operations we obtain a unitary cost of `~2.7`. We round up and obtain `3`. If we charge `3` per operation that costs `1` then we will have a remainder of `2` until we reach the next operation that costs `i`. Every time we reach an operation that costs `i` we will have enough surplus to pay for the operation.

    charge p step     => 3
    cost              => 1 | 2 | 1 | 4 | 1 | 1 | 1 | 8 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 16...
    credit after step => 2 | 3 | 5 | 6 | 8 |10 |12 | 7 | 9 |11 |12 |14 |16 |18 |20 | 7 ...
    
## Potential

In [45]:
def increment(a):
    i = 0
    while i < len(a) and a[i] == 1:
        a[i] = 0
        i = i + 1
    if i < len(a):
        a[i] = 1
        
bit_array = [0] * 8
print(0, '\t', bit_array)
for i in range(1, 17):
    increment(bit_array)
    print(i, '\t', bit_array[::-1])

0 	 [0, 0, 0, 0, 0, 0, 0, 0]
1 	 [0, 0, 0, 0, 0, 0, 0, 1]
2 	 [0, 0, 0, 0, 0, 0, 1, 0]
3 	 [0, 0, 0, 0, 0, 0, 1, 1]
4 	 [0, 0, 0, 0, 0, 1, 0, 0]
5 	 [0, 0, 0, 0, 0, 1, 0, 1]
6 	 [0, 0, 0, 0, 0, 1, 1, 0]
7 	 [0, 0, 0, 0, 0, 1, 1, 1]
8 	 [0, 0, 0, 0, 1, 0, 0, 0]
9 	 [0, 0, 0, 0, 1, 0, 0, 1]
10 	 [0, 0, 0, 0, 1, 0, 1, 0]
11 	 [0, 0, 0, 0, 1, 0, 1, 1]
12 	 [0, 0, 0, 0, 1, 1, 0, 0]
13 	 [0, 0, 0, 0, 1, 1, 0, 1]
14 	 [0, 0, 0, 0, 1, 1, 1, 0]
15 	 [0, 0, 0, 0, 1, 1, 1, 1]
16 	 [0, 0, 0, 1, 0, 0, 0, 0]


# Splay Trees

In [155]:
# Splay tree implementation in Java
# Author: AlgorithmTutor
# Tutorial URL: http://algorithmtutor.com/Data-Structures/Tree/Splay-Trees/

# data structure that represents a node in the tree

import sys

class Node:
    def  __init__(self, data):
        self.data = data
        self.parent = None
        self.left = None
        self.right = None

class SplayTree:
    def __init__(self):
        self.root = None

    def __print_helper(self, currPtr, indent, last):
        # print the tree structure on the screen
           if currPtr != None:
            sys.stdout.write(indent)
            if last:
                  sys.stdout.write("R----")
                  indent += "     "
            else:
                sys.stdout.write("L----")
                indent += "|    "

            print(currPtr.data)

            self.__print_helper(currPtr.left, indent, False)
            self.__print_helper(currPtr.right, indent, True)
    
    def __search_tree_helper(self, node, key):
        if node == None or key == node.data:
            return node

        if key < node.data:
            return self.__search_tree_helper(node.left, key)
        return self.__search_tree_helper(node.right, key)

    def __delete_node_helper(self, node, key):
        x = None
        t = None 
        s = None
        while node != None:
            if node.data == key:
                x = node

            if node.data <= key:
                node = node.right
            else:
                node = node.left

        if x == None:
            print("Couldn't find key in the tree")
            return
        
        # split operation
        self.__splay(x)
        if x.right != None:
            t = x.right
            t.parent = None
        else:
            t = None

        s = x
        s.right = None
        x = None

        # join operation
        if s.left != None:
            s.left.parent = None

        self.root = self.__join(s.left, t)
        s = None

    # rotate left at node x
    def __left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left != None:
            y.left.parent = x

        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    # rotate right at node x
    def __right_rotate(self, x):
        y = x.left
        x.left = y.right
        if y.right != None:
            y.right.parent = x
        
        y.parent = x.parent;
        if x.parent == None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y
        
        y.right = x
        x.parent = y

    # Splaying operation. It moves x to the root of the tree
    def __splay(self, x):
        while x.parent != None:
            if x.parent.parent == None:
                if x == x.parent.left:
                    # zig rotation
                    self.__right_rotate(x.parent)
                else:
                    # zag rotation
                    self.__left_rotate(x.parent)
            elif x == x.parent.left and x.parent == x.parent.parent.left:
                # zig-zig rotation
                self.__right_rotate(x.parent.parent)
                self.__right_rotate(x.parent)
            elif x == x.parent.right and x.parent == x.parent.parent.right:
                # zag-zag rotation
                self.__left_rotate(x.parent.parent)
                self.__left_rotate(x.parent)
            elif x == x.parent.right and x.parent == x.parent.parent.left:
                # zig-zag rotation
                self.__left_rotate(x.parent)
                self.__right_rotate(x.parent)
            else:
                # zag-zig rotation
                self.__right_rotate(x.parent)
                self.__left_rotate(x.parent)

    # joins two trees s and t
    def __join(self, s, t):
        if s == None:
            return t

        if t == None:
            return s

        x = self.maximum(s)
        self.__splay(x)
        x.right = t
        t.parent = x
        return x

    def __pre_order_helper(self, node):
        if node != None:
            sys.stdout.write(node.data + " ")
            self.__pre_order_helper(node.left)
            self.__pre_order_helper(node.right)

    def __in_order_helper(self, node):
        if node != None:
            self.__in_order_helper(node.left)
            sys.stdout.write(node.data + " ")
            self.__in_order_helper(node.right)

    def __post_order_helper(self, node):
        if node != None:
            self.__post_order_helper(node.left)
            self.__post_order_helper(node.right)
            std.out.write(node.data + " ")

    # Pre-Order traversal
    # Node->Left Subtree->Right Subtree
    def preorder(self):
        self.__pre_order_helper(self.root)

    # In-Order traversal
    # Left Subtree -> Node -> Right Subtree
    def inorder(self):
        self.__in_order_helper(self.root)

    # Post-Order traversal
    # Left Subtree -> Right Subtree -> Node
    def postorder(self):
        self.__post_order_helper(self.root)

    # search the tree for the key k
    # and return the corresponding node
    def search_tree(self, k):
        x = self.__search_tree_helper(self.root, k)
        if x != None:
            self.__splay(x)

    # find the node with the minimum key
    def minimum(self, node):
        while node.left != None:
            node = node.left
        return node

    # find the node with the maximum key
    def maximum(self, node):
        while node.right != None:
            node = node.right
        return node

    # find the successor of a given node
    def successor(self, x):
        # if the right subtree is not null,
        # the successor is the leftmost node in the
        # right subtree
        if x.right != None:
            return self.minimum(x.right)

        # else it is the lowest ancestor of x whose
        # left child is also an ancestor of x.
        y = x.parent
        while y != None and x == y.right:
            x = y
            y = y.parent
        return y

    # find the predecessor of a given node
    def predecessor(self, x):
        # if the left subtree is not null,
        # the predecessor is the rightmost node in the 
        # left subtree
        if x.left != None:
            return self.maximum(x.left)

        y = x.parent
        while y != None and x == y.left:
            x = y
            y = y.parent
        return y

    # insert the key to the tree in its appropriate position
    def insert(self, key):
        node =  Node(key)
        y = None
        x = self.root

        while x != None:
            y = x
            if node.data < x.data:
                x = x.left
            else:
                x = x.right

        # y is parent of x
        node.parent = y
        if y == None:
            self.root = node
        elif node.data < y.data:
            y.left = node
        else:
            y.right = node
        # splay the node
        self.__splay(node)

    # delete the node from the tree
    def delete_node(self, data):
        self.__delete_node_helper(self.root, data)

    # print the tree structure on the screen
    def pretty_print(self):
        self.__print_helper(self.root, "", True)

def assignment_3():
    # 2, 1, 4, 5, 9, 3, 6, 7
    tree = SplayTree()
    tree.insert(2)
    tree.insert(1)
    tree.insert(4)
    tree.insert(5)
    tree.insert(9)
    tree.insert(3)
    tree.insert(6)
    tree.insert(7)
    tree.pretty_print()
    
def practice_quiz_u3_four():
    # must use successors 
    # 5, 3, 8, 4, 6, 10, 7, 9
    tree = SplayTree()
    tree.insert(9)
    tree.insert(7)
    tree.insert(4)
    
    tree.insert(6)
    tree.insert(10)
    
    tree.insert(3)
    tree.insert(8)
    tree.insert(5)
    # DONT CHANGE THE INSERT ORDER
    tree.pretty_print()
    print(tree.delete_node(8))
    tree.pretty_print()
    
def practice_quiz_u3_five():
    # 7, 4, 11, 2, 6, 9, 12, 3, 5,8, 10
    tree = SplayTree()
    tree.insert(8)
    tree.insert(3)
    tree.insert(5)
    tree.insert(12)
    tree.insert(10)
    tree.insert(2)
    tree.insert(9)
    tree.insert(6)
    tree.insert(4)
    tree.insert(11)
    tree.insert(7)
    # DONT CHANGE THE INSERT ORDER
    tree.pretty_print()
    print(tree.search_tree(5))
    tree.pretty_print()

def practice_quiz_u3_six():
    # 11, 5, 28,3, 9, 22, 2, 4, 7, 20, 25,6, 8
    tree = SplayTree()
    
    tree.insert(6)
    
    tree.insert(20)
    tree.insert(8)
    
    tree.insert(2)
    tree.insert(4)
    tree.insert(7)
    
    tree.insert(3)
    
    tree.insert(9)
    tree.insert(25)
    tree.insert(22)
    
    tree.insert(5)
    tree.insert(28)
    tree.insert(11)
    # DONT CHANGE THE INSERT ORDER
    tree.pretty_print()
    print(tree.delete_node(8))
    tree.pretty_print()

    # fourth question
# 7 , 10, 9
# 9, 10 (correct)

# 3 + 1-1 p
# 1 + 3* p
# 3 + 3* q
# 3 + 1 -1 q (correct)

#true, false
practice_quiz_u3_five()
print('#############')
practice_quiz_u3_six()

R----7
     L----4
     |    L----2
     |    |    R----3
     |    R----6
     |         L----5
     R----11
          L----9
          |    L----8
          |    R----10
          R----12
None
R----5
     L----4
     |    L----2
     |    |    R----3
     R----7
          L----6
          R----11
               L----9
               |    L----8
               |    R----10
               R----12
#############
R----11
     L----5
     |    L----3
     |    |    L----2
     |    |    R----4
     |    R----9
     |         L----7
     |         |    L----6
     |         |    R----8
     R----28
          L----22
          |    L----20
          |    R----25
None
R----7
     L----5
     |    L----3
     |    |    L----2
     |    |    R----4
     |    R----6
     R----11
          L----9
          R----28
               L----22
               |    L----20
               |    R----25


# Quiz 3

In [158]:
def is_power_of_three(n):
    return 1162261467 % n == 0

count = []
sum_ones = 0
sum_non_ones = 0
for i in range(1,28):
    if is_power_of_three(i):
        count.append(i)
        sum_non_ones += i
    else:
        count.append(1)
        sum_ones += 1


print(" | ".join(map(str, count)))
print(sum_ones, sum_non_ones)

1 | 1 | 3 | 1 | 1 | 1 | 1 | 1 | 9 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 1 | 27
23 40


In [167]:
def quiz_u3_five():
    # 7, 4, 11, 2, 6, 9, 12, 3, 5, 8, 10
    tree = SplayTree()
    
    tree.insert(12)
    tree.insert(3)
    tree.insert(5)
    tree.insert(8)
    tree.insert(10)
    
    tree.insert(6)
    tree.insert(2)
    tree.insert(9)
    
    tree.insert(4)
    tree.insert(11)
    tree.insert(7)
    # DONT CHANGE THE INSERT ORDER
    tree.pretty_print()
    print(tree.search_tree(9))
    tree.pretty_print()

quiz_u3_five()

R----7
     L----4
     |    L----2
     |    |    R----3
     |    R----6
     |         L----5
     R----11
          L----9
          |    L----8
          |    R----10
          R----12
None
R----9
     L----7
     |    L----4
     |    |    L----2
     |    |    |    R----3
     |    |    R----6
     |    |         L----5
     |    R----8
     R----11
          L----10
          R----12


In [192]:
def quiz_u3_six():
    # 11, 5, 28, 3, 9, 22, 2, 4, 7, 20, 25, 6, 8
    tree = SplayTree()
    
    tree.insert(20)
    tree.insert(6)
    tree.insert(8)
    
    tree.insert(2)
    
    tree.insert(7)
    tree.insert(4)
    
    tree.insert(3)
    tree.insert(9)
    tree.insert(22)
    tree.insert(25)
    
    tree.insert(5)
    tree.insert(28)
    tree.insert(11)
    # DONT CHANGE THE INSERT ORDER
    tree.pretty_print()
    print(tree.delete_node(7))
    tree.pretty_print()

quiz_u3_six()

R----11
     L----5
     |    L----3
     |    |    L----2
     |    |    R----4
     |    R----9
     |         L----7
     |         |    L----6
     |         |    R----8
     R----28
          L----22
          |    L----20
          |    R----25
None
R----6
     L----5
     |    L----3
     |    |    L----2
     |    |    R----4
     R----11
          L----9
          |    L----8
          R----28
               L----22
               |    L----20
               |    R----25
